In [9]:
import sys
import os
import time
import operator
import pandas as pd

class Node():
    def __init__(self,count=0):
        self.count=count
    def add_count(self,count=1):
        self.count+=count
    def __str__(self):
        return 'ID: {}, count: {}'.format(self.ID,self.count)
    def __repr__(self):
        return ''

class Head(Node):
    def __init__(self):
        super().__init__()
        self.distinct = hyperloglog.HyperLogLog(0.01)
    def __str__(self):
        return 'total count: {}, distinct element: {}'.format(self.count,len(self.distinct))
    def __repr__(self):
        return '[count: {}, distinct: {}]'.format(self.count,len(self.distinct))

class Tail(Node):
    def __init__(self,ID,count):
        self.ID = ID
        super().__init__(count)
    def __str__(self):
        return 'ID: {}, count: {}'.format(self.ID,self.count)
    def __repr__(self):
        return "'{}', count: {}".format(self.ID,self.count)

def find(e,element_list):
    try:
        index = [ele.ID for ele in element_list].index(e.ID)
    except:
        index=-99
    return index  
    
    
start=time.time()

filename='caida_0.dat'
filepath="..\\dataset\\"
src_data=os.path.join(filepath,filename)

size=820
Top=[]
#item_count=100000

with open(src_data,'rb') as file:
     #以binary讀取，資料型態也為byte
    while True:
        e=str(file.read(13))
        if len(e)<13:
            print('EOF')
            break
        else:
            item=Tail(e,1)
            #item_count-=1
            # print("read {}th element: {}".format(item_count,element))
            if len(Top)==0:
                Top.append(item)
            else:
                index=find(item,Top)
                if index<0:
                    if len(Top)<size:
                        Top.append(item)
                    else:
                        # replace last element with count 
                        Top[-1].ID=item.ID
                        Top[-1].count+=1
                else:
                    Top[index].count+=1
            Top.sort(key=operator.attrgetter('count'),reverse=True)

end=time.time()
print(Top[:20],len(Top))
print("Total memory {0} bytes :Top-{1} with size {0} bytes.".format(sys.getsizeof(Top),size))
print("Execution time:{} seconds.".format(str(end-start)))


#====================result tocsv=============================
templi=[[i.ID,i.count] for i in Top]

df=pd.DataFrame(templi,columns=['ID', 'Count'])
name="SS_caida0"+'_'+str(size)
df.to_csv(name+".csv",index=False)


#====================result compare=============================
path="..\\dataset\\"
groundtruth='caida_0_ground_truth.csv'
final=name+".csv"

grtruth=pd.read_csv(groundtruth)
My_result=pd.read_csv(final)


# precision
gt_set=set(grtruth['Element'][:size])
    # Top-size of ground truth
my_set=set(My_result['ID'])
precision=len(gt_set & my_set)/len(my_set)
    # &: set 交集運算

# ARE/AAE
gt_li=grtruth.values.tolist()[:size]
top_li=My_result.values.tolist()
ID=[j[0] for j in gt_li]
top_are=0
top_aae=0
tp=0
fp=0
for item in top_li:
    if item[0] in ID:
        index=ID.index(item[0])
        tp+=1
        top_are+=abs(gt_li[index][1]-item[1])/gt_li[index][1]
        top_aae+=abs(gt_li[index][1]-item[1])
    else:
        fp+=1
top_are=top_are/size
top_aae=top_aae/size

print("Find:{}, TP:{}, FP:{}".format(len(gt_set & my_set),tp,fp))
print("Precision: {:8.4f}".format(precision))
print("ARE: {:8.6f}".format(top_are))
print("AAE: {:8.6f}".format(top_aae))    


EOF
['b'\xd2\x1d9\xfa\x00P\x0f\x13I\x9a\xf89\x06'', count: 16889, 'b'E.\x1c)\xa4\xb2\x89\xb6\x06\xba\xd2\xad\x06'', count: 11647, 'b'V+x\xe8\x00Po\xcd\xe4\xde\xccw\x06'', count: 10113, 'b'\xb7\xf6h\xd0\xc1\x8b+\xfc\xe1\x08\x00P\x06'', count: 9089, 'b'\x01l|\xff\x8f\r\x01l\xc5\xc8\x01\xbb\x06'', count: 8477, 'b'\x99\xc1\xe9Z\x00P+\xfc\xe0=\xe7%\x06'', count: 7826, 'b'c\x0f\xb2\xd4\x80\xd2\x01`\x8d\x12\x01\xbb\x06'', count: 6400, 'b'\xd2\x1e\xbeu\x01\xbb+\xfc\xe0.\x92\xd5\x06'', count: 5777, 'b'\x01l|\xec\xcc\x1b\x01l\xc5\xc8\x01\xbb\x06'', count: 5421, 'b'E\x0c\xd5\xf7\xd7(\xdd.\xdc\xec\x00P\x06'', count: 5199, 'b'E\x0c\xd5\xf5\xa4\x14\xdd.\xdc\xec\x00P\x06'', count: 5087, 'b'OZ\x0e\xae\xdc\xf1o\xcd\x19\xf0\x1f\x90\x11'', count: 4621, 'b'E\x0c\xd1K\x85\xf6\xdd.\xdc\xf2\x00P\x06'', count: 4485, 'b'\x01l$\xce\x0e\xad\x01l\xc5\xc6\x01\xbb\x06'', count: 4447, 'b'\x99\xc1\x8f\x05\x00P+\xfc\xe2#\xa0W\x06'', count: 4401, 'b'\x01l|\xecJ\x97\x01l\xc3i\x01\xbb\x06'', count: 4386, 'b'\xdd\x01b\xdf